In [4]:
def load_model(model_path):
    model = BertForSequenceClassification.from_pretrained('personalized-learning-model')
    return model

def average_models(model_paths):
    # Load the models
    models = [load_model(path) for path in model_paths]

    # Initialize the averaged model with the same configuration as the first model
    averaged_model = BertForSequenceClassification.from_pretrained('personalized-learning-model')

    # Average the parameters
    with torch.no_grad():
        for name, param in averaged_model.named_parameters():
            # Sum the parameters from each model
            temp = sum(model.state_dict()[name] for model in models)
            # Divide by the number of models to get the average
            param.copy_(temp / len(models))

    return averaged_model

# Paths to your saved models
model_path1 = 'personalized-learning-model'
model_path2 = 'averaged-global-model'

# Average the models
averaged_model = average_models([model_path1, model_path2])

# Save the averaged model (if needed)
averaged_model.save_pretrained('averaged-global-model')

In [8]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

# Inference Function
def calculate_averaged_learning_measures(logins, time_spent, page_visits, search_queries, activity_completion, quiz_score, reactions_pos, reactions_neg, feedback):
    input_text = f"{logins} {time_spent} {page_visits} {search_queries} {activity_completion} {quiz_score} {reactions_pos} {reactions_neg} {feedback}"
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('averaged-global-model')
    tokens = tokenizer(input_text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    tokens = {key: value.to(device) for key, value in tokens.items()}
    with torch.no_grad():
        output = model(**tokens)
        scores = torch.sigmoid(output.logits).squeeze(0).cpu().numpy()

    return {
        'Conscientiousness': round(scores[0] * 10, 2),
        'Motivation': round(scores[1] * 10, 2),
        'Understanding': round(scores[2] * 10, 2),
        'Engagement': round(scores[3] * 10, 2)
    }

# Example call to the function to demonstrate its use
averaged_learning_scores = calculate_learning_measures(4, 9, 11, 5, 80.0, 84.0, 3, 2, 6)
print("Learning Measures Scores:", averaged_learning_scores)

Learning Measures Scores: {'Conscientiousness': 5.16, 'Motivation': 6.17, 'Understanding': 5.98, 'Engagement': 6.17}
